In [ ]:
!pip install flair

     |████████████████████████████████| 284 kB 14.5 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 23.2 MB/s 
     |████████████████████████████████| 981 kB 70.2 MB/s 
     |████████████████████████████████| 788 kB 61.7 MB/s 
     |████████████████████████████████| 2.6 MB 70.1 MB/s 
     |████████████████████████████████| 19.7 MB 71 kB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 776.8 MB 16 kB/s 
     |████████████████████████████████| 62 kB 931 kB/s 
     |████████████████████████████████| 3.3 MB 76.7 MB/s 
     |████████████████████████████████| 636 kB 79.1 MB/s 
     |████████████████████████████████| 895 kB 51.3 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9705 sha256=01644c50ca1481b27581989fa9b64f1295cced18edc6

In [ ]:
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings
import numpy as np
import torch
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
embedding_model = TransformerDocumentEmbeddings('bert-large-uncased', fine_tune = False)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def read_comments_from_dataset(filename):
  return pd.read_csv(filename)[["commentID", "commentBody", "commentSequence", "articleID", "parentID"]]

In [ ]:
filenames = ["CommentsMay2017.csv"]

In [ ]:
comment_sets = [(filename, read_comments_from_dataset("gdrive/MyDrive/Research/NewYorkTimesComments/" + filename)) for filename in filenames]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
len(comment_sets)

1

In [ ]:
from tqdm import tqdm

def compute_embeddings(comments):
  embeddings = []

  count = 0
  embedding_model = TransformerDocumentEmbeddings('bert-large-uncased', fine_tune = False)

  for comment in tqdm(comments.itertuples()):
    try:
      comment_sentence = Sentence(comment.commentBody)
      embedding_model.embed(comment_sentence)
    except:
      comment_sentence = Sentence("None")
      embedding_model.embed(comment_sentence)
    embeddings.append({"ID": comment.commentID,
                      "article_id": comment.articleID, 
                      "commentBody": comment.commentBody,
                      "embedding": comment_sentence.embedding})
    count += 1
    if count > 5000:
      del embedding_model
      del comment_sentence
      embedding_model = TransformerDocumentEmbeddings('bert-large-uncased', fine_tune = False)
      count = 0
  return embeddings

In [ ]:
for comment_set in comment_sets:
  embedding = compute_embeddings(comment_set[1])
  torch.save(embedding, "gdrive/MyDrive/Research/embedded_comments_" + comment_set[0] + "_bert-large-uncased.pt")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
4997it [02:10, 36.92it/s]Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'c